In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import warnings
warnings.filterwarnings('ignore')
import h5py

class NeuralNetwork:
    
    def __init__(self):
         # Learning Rate
        self.l_rate = 0.001
         # Total iterations
        self.epochs = 1000
    

    def gradient_descent(self, x_train_data, y_train_data):
        
        w = np.random.rand(x_train_data.shape[1], 1) * 0.01 # Weights matrix = ()
        b = np.zeros((1, 1), dtype='float') # Bais matrix = (1 x 1)

        m = len(x_train_data) # length of train data set
    
        for i in range(self.epochs):
#             print(i)
            ######## Forward Propagation ########
            # Hypothesis function :
            z = (np.dot(w.T, x_train_data.T)) + b # Z = W*X = (1 x m)
            
            # Activation Function :
            a =  1 / 1 + np.exp(-z) # Activation function = sigmoid(-z) = (1 x m)

            ####### Backward Propagation ########
            dz = (a - y_train_data.T) # dZ = a - y
            dw = (np.dot(x_train_data.T, dz.T)) / m # dW = X*T
            db = (np.sum(dz)) / m # db = dZ
            
             # Gradient Descent :
            w = w - np.dot(self.l_rate, dw)  # w = w - alpha * dW
            b = b - np.dot(self.l_rate, db)  # b = b - alpha * db             
        return w, b
    
    # Prediction :
    def prediction(self, parameters, x_test_data):
        z = np.dot(parameters[0].T, x_test_data.T) + parameters[1] # Z = W*X = (1 x m)
        return 1 / 1 + np.exp(-z) #  Activation function = sigmoid(-z) = (1 x m)
    
    def categorical_data_handling(self, df):
        categorical_data = []
        for name in list(df.columns):
            if is_numeric_dtype(df[name]):
                pass
            else:
                categorical_data.append(name)
        return pd.get_dummies(df, columns=categorical_data)
    
def main():   
 
    # Create a class object
    nn = NeuralNetwork()

    df = pd.read_csv("bank.csv", delimiter = ";")

    df.y.replace(('yes', 'no'), (1, 0), inplace=True)
    
    df = nn.categorical_data_handling(df)

    y_data_set = pd.DataFrame(df.y)
    x_data_set = df.drop(["y"], axis=1)
    
    train_size = int(np.ceil((len(y_data_set) * 80) / 100)) # Divide the data in ratio 80:20
    test_size = int(len(y_data_set) - train_size)

    x_train_data = np.array(x_data_set[:train_size])
    y_train_data = (np.array(y_data_set[:train_size])).T
    x_test_data = np.array(x_data_set[train_size:])
    y_test_data = (np.array(y_data_set[train_size:])).T  
    

    # Create a class object
    nn = NeuralNetwork()
    
    # parameter stores the value of weights and bais
    parameters = nn.gradient_descent(x_train_data, y_train_data)
    
    # pred stores the predicted values from test data set
    pred = np.nan_to_num(nn.prediction(parameters, x_test_data)) 
    
     # pred stores the predicted values from train data set
    pred_train= np.nan_to_num(nn.prediction(parameters, x_train_data) ) 
    
    # It calculates the accuracy

    train_accuracy = (100 - np.mean(np.abs(pred_train - y_train_data)) * 100)
    test_accuracy = (100 - np.mean(np.abs(pred - y_test_data)) * 100)

    print("train_accuracy = ", train_accuracy)
    print("test_accuracy = ", test_accuracy)
    
    

if __name__ == '__main__':
    main()